In [3]:
import pandas as pd
import os
import numpy as np
import folium
import h3
import matplotlib.pyplot as plt
from matplotlib.colors import rgb2hex

p = os.path.join

In [4]:
cmap = plt.get_cmap('Greys')

In [6]:
df = pd.read_csv(p('..', 'prediction', 'final_prediction.csv'), index_col=0)

In [8]:
cities = ['Lima', 'Montevideo', 'Quito','Sao Paulo']

In [11]:
df.city == 'Lima'

898e62d58c3ffff     True
898e62c0617ffff     True
898e62d523bffff     True
898e62d5353ffff     True
898e62d0ebbffff     True
                   ...  
89a8100e837ffff    False
89a8100c307ffff    False
89a8100e333ffff    False
89a8100e56bffff    False
89a8100c593ffff    False
Name: city, Length: 996, dtype: bool

In [19]:
for c in cities:
    city_df = df[df.city == c]
    ring = {r for h in city_df.index for r in h3.k_ring(h, 3)}
    latlons = [h3.h3_to_geo(r) for r in ring]
    lat, lon = np.array(latlons).mean(axis=0)
    m = folium.Map(location=[lat, lon])
    m.fit_bounds(latlons)

    for r in ring:
        try:
            level = city_df.loc[r]['NO2_adjusted']
        except KeyError:
            continue
        folium.Polygon(
            h3.h3_to_geo_boundary(r),
            tooltip=f'{np.round(level, 1)} ppb',
            fill_color= rgb2hex(cmap(level/city_df['NO2_adjusted'].max())), 
            color='black',
            fill_opacity= 0.6
        ).add_to(m)
    m.save(f'maps/{c}.html')